In [1]:
suppressMessages(library(rwwa))
png_res <- 240

# Functions

## Automate synthesis

In [34]:
full_synthesis <- function(res_obs, res_mdl, dI_type = "abs", cf_nm = "pi") {
    
    # when adding to rwwa package, make column names an optional argument but use these defaults if missing
    # also should add the unweighted values to the tables

    # Synthesise change in intensity
    obs_dI <- res_obs[,grepl(paste0(dI_type, ".", cf_nm, "_"), colnames(res_obs))]
    mdl_dI <- res_mdl[,grepl(paste0("attr_dI.",dI_type,".",cf_nm, "_"), colnames(res_mdl))]

    attr_dI <- synthesis(obs_dI, mdl_dI, synth_type = dI_type)$df
    proj_dI <- synthesis(obs_in = NA, res_mdl[,grepl(paste0("proj_dI.",dI_type), colnames(res_mdl))], synth_type = dI_type)$df

    # Synthesise PR
    obs_pr <- infer_infinite(res_obs[,grepl(paste0("PR.", cf_nm), colnames(res_obs))])
    mdl_pr <- infer_infinite(res_mdl[,grepl("attr_PR", colnames(res_mdl))])
    
    attr_pr <- synthesis(obs_pr, mdl_pr, synth_type = "PR")$df
    proj_pr <- synthesis(obs_in = NA, infer_infinite(res_mdl[,grepl("proj_PR", colnames(res_mdl))]), synth_type = "PR")$df


    # compile the results of the attribution analysis
    tbl_attr <- t(sapply(c("obs_synth", "model_synth", "synth"), function(cnm) {
        di = signif(attr_dI[attr_dI$group == cnm,c("est", "lower", "upper")], 3)
        pr = signif(attr_pr[attr_pr$group == cnm,c("est", "lower", "upper")], 3)
        
        sapply(list("di" = di, "pr" = pr), function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    }))

    # compile projected results
    tbl_proj <- t(sapply(c("model_synth"), function(cnm) {
        di = signif(proj_dI[proj_dI$group == cnm,c("est", "lower", "upper")], 3)
        pr = signif(proj_pr[proj_pr$group == cnm,c("est", "lower", "upper")], 3)
        
        sapply(list("di" = di, "pr" = pr), function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    }))

    # return single table
    return(list("dI_attr" = attr_dI, "dI_proj" = proj_dI, "PR_attr" = attr_pr, "PR_proj" = proj_pr,
                "table" = rbind(tbl_attr, tbl_proj)[,c("pr", "di")]))
}

## Clean up plotting

In [3]:
# modified plotting function with butt ends, instead of square
plot_synthesis <- function (synth, xlim, lwd = 10, xlab = "", main = "", add_space = T, 
    log = NA, hide_labels = F) {
    gcols = c(obs = adjustcolor("blue", 0.5), obs_synth = "blue", 
        models = adjustcolor("red", 0.5), model_synth = "red", 
        synth = "magenta")
    if (is.na(log)) {
        if (!is.null(synth$synth)) {
            if (synth$synth_type == "PR") {
                logaxs <- "x"
            }
            else {
                logaxs <- ""
            }
        }
        else {
            logaxs <- ""
        }
    }
    else {
        if (log) {
            logaxs <- "x"
        }
        else {
            logaxs <- ""
        }
    }
    if (is(synth, "list")) 
        synth <- synth$df
    if (missing(xlim)) {
        if (logaxs == "x") {
            xlim <- exp(range(pretty(log(as.numeric(unlist(synth[, 
                c("lower", "upper", "l_wb", "u_wb")]))))))
        }
        else {
            xlim <- range(pretty(as.numeric(unlist(synth[, c("lower", 
                "upper", "l_wb", "u_wb")]))))
        }
    }
    if (is.numeric(synth$group)) 
        synth$group <- names(gcols)[synth$group]
    nobs <- sum(synth$group == "obs")
    nmod <- sum(synth$group == "models")
    if (add_space & nobs > 0) {
        yy <- c(rev(0:nobs + nmod + 4), rev(0:nmod + 2), 0)
    }
    else {
        yy <- nrow(synth):1
    }
    if (logaxs == "x") {
        vline <- 1
    }
    else {
        vline <- 0
    }
    plot(0, type = "n", xlim = xlim, ylim = range(yy) + c(-0.5, 
        0.5), log = logaxs, yaxt = "n", ylab = "", xlab = xlab, 
        main = main)
    grid(ny = NA, col = adjustcolor("black", 0.1), lty = 1)
    abline(v = vline, lty = 1)
    gcols <- gcols[synth$group]
    segments(y0 = yy, x0 = synth$l_wb, x1 = synth$u_wb, lwd = lwd + 2, col = "black", lend = 1)
    segments(y0 = yy, x0 = synth$l_wb, x1 = synth$u_wb, lwd = lwd - 2, col = "white", lend = 1)
    segments(y0 = yy, x0 = synth$lower, x1 = synth$upper, lwd = lwd, 
        col = gcols, lend = 1)
    points(synth$est, yy, pch = 21, bg = gcols, lwd = 2, cex = lwd/10)
    if (!hide_labels) 
        axis(2, at = yy, labels = synth$model, las = 1)
}

# Synthesis before evaluation

In [35]:
varnm <- "rx1day-ondjfm"
rnm <- "s"



In [56]:
# variables that don't need to be changed in this case
mtype <- "gmst+nao"
dI_type = "rel"# list("hdwi" = "abs", "pr-ndj" = "rel")[[varnm]]
cf_nm = "pi"

# load & clean all the data
res_obs <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")
res_obs <- res_obs[sapply(strsplit(rownames(res_obs), "_"), "[", 2) == mtype,]
rownames(res_obs) <- sapply(strsplit(rownames(res_obs), "_"), "[", 1)

res_mdl <- read.csv(paste0("res-cordex_",varnm,"_",rnm,"_",mtype,".csv"), row.names = "X.1")
# res_mdl <- res_mdl[res_mdl$include == "Y",]

# run synthesis
synth <- full_synthesis(res_obs, res_mdl, dI_type = dI_type)

# overwrite plotting limits
xlim_PR <- c(1e-3, 1e+3)

xlim_dI <- range(pretty(unlist(rbind(synth$dI_attr, synth$dI_proj)[,3:7])))

# # overwrite plotting limits
# xlim_PR <- list("rx1day-ondjfm_s" = c(0.1, 100),
#                 "rx1day-ondjfm_n" = c(0.1, 100))[[paste0(varnm, "_", rnm)]]

# xlim_dI <- list("rx1day-ondjfm_s" = c(-1, 3),
#                 "rx1day-ondjfm_n" = c(-5, 15))[[paste0(varnm, "_", rnm)]]

In [57]:
# save all tables
write.csv(synth$dI_attr, paste0("synth/synth-attr-dI_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)
write.csv(synth$PR_attr, paste0("synth/synth-attr-PR_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)

write.csv(synth$dI_proj, paste0("synth/synth-proj-dI_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)
write.csv(synth$PR_proj, paste0("synth/synth-proj-PR_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)

# table of results to paste into document
write.csv(synth$table, paste0("synth/synth-table_",varnm,"_",rnm,"_",mtype,".csv"))



# figures for past change
png(paste0("fig/synth-attr_",varnm,"_",rnm,"_",mtype,".png"), height = png_res * 7/4, width = png_res * 3); {
    prep_window(c(1,2), oma = c(0,20,0,0), res = 140, mar = c(3,1,2,1), h = 7, w = 5)
    plot_synthesis(synth$PR_attr, hide_labels = F, main = "(a) Probability ratio", xlim = xlim_PR, log = T)
    plot_synthesis(synth$dI_attr, hide_labels = T, main = "(b) Change in intensity", xlim = xlim_dI)
    
    # mtext(paste0(varnm, " - ", rnm), outer = T, side = 3, font = 2)
}; dev.off()

# figures for future change
png(paste0("fig/synth-proj_",varnm,"_",rnm,"_",mtype,".png"), height = png_res * 7/4, width = png_res * 3); {
    prep_window(c(1,2), oma = c(0,20,0,0), res = 140, mar = c(3,1,2,1), h = 7, w = 5)
    plot_synthesis(synth$PR_proj, hide_labels = F, main = "(a) Probability ratio", xlim = xlim_PR, log = T)
    plot_synthesis(synth$dI_proj, hide_labels = T, main = "(b) Change in intensity", xlim = xlim_dI)
    
    # mtext(paste0(varnm, " - ", rnm), outer = T, side = 3, font = 2)
}; dev.off()

agg_record_378402524 
                   2

agg_record_378402524 
                   2

# Initial synthesis plots

In [ ]:
varnm <- "hdwi"
mtype <- "gmst-only"
rnm <- "patagonia"

res_obs <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[mtype,,drop = F]
res_mdl <- read.csv(paste0("res-cordex_",varnm,"_",mtype,"_",rnm,".csv"), row.names = "X.1")

,X,eval_cor.x.gmst_est,eval_cor.x.gmst_lower,eval_cor.x.gmst_upper,eval_cor.x.nao_est,eval_cor.x.nao_lower,eval_cor.x.nao_upper,eval_cor.gmst.nao_est,eval_cor.gmst.nao_lower,eval_cor.gmst.nao_upper,⋯,proj_dI.rel.fut_upper,aic_eval,aic_attr,aic_proj,attr_nobs,attr_nsamp,attr_nfailed,proj_nobs,proj_nsamp,proj_nfailed
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
EUR-11_CanESM2_r1i1p1_CCLM4-8-17,x ~ gmst + nao (rp10),0.10834242,-0.1789483992,0.3691395,-0.414792124,-0.6557973,-0.13157749,-0.14002653,-0.4382735,0.16726090,⋯,2.8921714,306.6260,469.9104,738.7138,77,1000,0,121,1000,0
EUR-11_CanESM2_r1i1p1_REMO2015,x ~ gmst + nao (rp10),-0.01380565,-0.2530435043,0.2182477,-0.411380009,-0.6013179,-0.19506322,-0.14002653,-0.4382735,0.16726090,⋯,4.7352620,325.4363,505.5304,801.9373,77,1000,0,121,1000,0
EUR-11_CNRM-CM5_r1i1p1_ALADIN53,x ~ gmst + nao (rp10),0.23607135,-0.0454329980,0.4951960,-0.185581462,-0.4154316,0.07112259,-0.21904579,-0.4466774,0.01747289,⋯,12.0579219,312.0970,503.7887,784.4392,77,1000,0,121,1000,0
EUR-11_CNRM-CM5_r1i1p1_ALADIN63,x ~ gmst + nao (rp10),0.02621232,-0.2948478135,0.3403613,-0.281130767,-0.5089120,-0.01454705,-0.21904579,-0.4466774,0.01747289,⋯,12.2127293,313.3875,501.2298,801.0899,76,1000,0,120,1000,0
EUR-11_CNRM-CM5_r1i1p1_ALARO-0,x ~ gmst + nao (rp10),0.12103110,-0.1651464410,0.3773577,-0.126090106,-0.4039282,0.18300892,-0.21904579,-0.4449936,0.01747289,⋯,9.5933680,329.7007,520.5398,821.9132,77,1000,0,121,1000,0
EUR-11_CNRM-CM5_r1i1p1_CCLM4-8-17,x ~ gmst + nao (rp10),0.19201391,-0.0840407676,0.4105991,0.005320606,-0.1995098,0.22464850,-0.21904579,-0.4466774,0.01747289,⋯,9.1423064,301.2464,477.5096,774.8612,77,1000,0,121,1000,0
EUR-11_CNRM-CM5_r1i1p1_COSMO-crCLIM-v1-1,x ~ gmst + nao (rp10),0.07867095,-0.1944734314,0.3617479,-0.189405037,-0.4077010,0.04074475,-0.21904579,-0.4449936,0.01747289,⋯,10.3261581,319.5188,508.7877,792.4726,76,1000,0,120,1000,0
EUR-11_CNRM-CM5_r1i1p1_HadREM3-GA7-05,x ~ gmst + nao (rp10),0.07484680,-0.2302165140,0.3799754,-0.216307604,-0.4410984,0.01417500,-0.21904579,-0.4466774,0.01330644,⋯,11.0865917,344.3807,521.4830,836.2516,75,1000,0,119,1000,0
EUR-11_CNRM-CM5_r1i1p1_HIRHAM5,x ~ gmst + nao (rp10),0.11162992,-0.1680653853,0.3764747,-0.211263940,-0.4256345,0.01904139,-0.21904579,-0.4466774,0.01747289,⋯,11.4445622,336.9711,533.9072,839.4719,76,1000,0,120,1000,0


In [42]:
if (varnm %in% c("pr-ndj")) {
    obs_dI <- res_obs[,grepl("dI.rel.pi", colnames(res_obs))]
    mdl_dI <- res_mdl[,grepl("attr_dI.rel", colnames(res_mdl))]
    synth_dI <- synthesis(obs_dI, mdl_dI, synth_type = "rel")

    proj_dI <- synthesis(obs_in = NA, res_mdl[,grepl("proj_dI.rel", colnames(res_mdl))], synth_type = "rel")
    
} else {
    obs_dI <- res_obs[,grepl("dI.abs.pi", colnames(res_obs))]
    mdl_dI <- res_mdl[,grepl("attr_dI.abs", colnames(res_mdl))]
    synth_dI <- synthesis(obs_dI, mdl_dI, synth_type = "abs")

    proj_dI <- synthesis(obs_in = NA, res_mdl[,grepl("proj_dI.abs", colnames(res_mdl))], synth_type = "abs")
    
}

obs_pr <- res_obs[,grepl("PR.pi", colnames(res_obs))]
mdl_pr <- infer_infinite(res_mdl[,grepl("attr_PR", colnames(res_mdl))])
synth_pr <- synthesis(obs_pr, mdl_pr, synth_type = "PR")

proj_pr <- synthesis(obs_in = NA, infer_infinite(res_mdl[,grepl("proj_PR", colnames(res_mdl))]), synth_type = "PR")

In [43]:
write.csv(synth_dI$df, paste0("synth/synth-attr-dI_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)
write.csv(synth_pr$df, paste0("synth/synth-attr-PR_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)

write.csv(proj_dI$df, paste0("synth/synth-proj-dI_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)
write.csv(proj_pr$df, paste0("synth/synth-proj-PR_",varnm,"_",rnm,"_",mtype,".csv"), row.names = F)

In [32]:
png(paste0("fig/synth-attr_",varnm,"_",rnm,"_",mtype,".png"), height = png_res * 7/4, width = png_res * 3); {
    prep_window(c(1,2), oma = c(0,20,2,0), res = 140, mar = c(3,1,1,1), h = 7, w = 5)
    plot_synthesis(synth_pr, hide_labels = F)
    plot_synthesis(synth_dI, hide_labels = T)
    
    mtext(paste0(varnm, " - ", rnm), outer = T, side = 3, font = 2)
}; dev.off()

agg_record_441618038 
                   2

In [33]:
png(paste0("fig/synth-proj_",varnm,"_",rnm,"_",mtype,".png"), height = png_res * 7/4, width = png_res * 3); {
    prep_window(c(1,2), oma = c(0,20,2,0), res = 140, mar = c(3,1,1,1), h = 7, w = 5)
    plot_synthesis(proj_pr, hide_labels = F)
    plot_synthesis(proj_dI, hide_labels = T)
    
    mtext(paste0(varnm, " - ", rnm), outer = T, side = 3, font = 2)
}; dev.off()

agg_record_1553383004 
                    2